In [1]:
import musicbrainzngs
from musicbrainzngs import WebServiceError

musicbrainzngs.set_useragent("Nouserentered Song Analysis Project", "0.1.0", contact="matthew.shu@yale.edu"
                             )

In [188]:
from difflib import SequenceMatcher

# Modified from: https://stackoverflow.com/a/17388505
def match(orig_name, found_name):
    return SequenceMatcher(None, orig_name, found_name).ratio() > .7

In [189]:
def search_artist(artist_id: int, name: str):
    try:
        result = musicbrainzngs.search_artists(name)
    except WebServiceError as exc:
        print("Something went wrong with the request: %s" % exc)
    else:
        artist = result["artist-list"]
        if len(artist) != 0 and match(artist[0]["name"], name):
            # Currently just takes the top artist, isn't always great if there are multiple artists!
            top_artist = artist[0]
            top_artist["artist_id"] = artist_id
            return top_artist
        else:
            return {"name": name, "artist_id": artist_id}
def search_artist_series(artist_id: int, name: str):
    artist_data = search_artist(artist_id, name)
    if artist_data:
        return pd.json_normalize(artist_data).iloc[0]
    else:
        return

# Load Data

In [190]:
%load_ext dotenv
%dotenv

import os
import sqlite3
import pandas as pd

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [191]:
SQL_FILEPATH = os.getenv("PLAYBACK_FILE")
lyrics_db = sqlite3.connect(SQL_FILEPATH)

df = pd.read_sql_query("SELECT * FROM lyrics", lyrics_db)
print(df.head())

   index  track_id  year              artist  \
0      0         0  1944           Red Foley   
1      1         1  1944  The King Cole Trio   
2      2         2  1944        Louis Jordan   
3      3         3  1944          Tex Ritter   
4      4         4  1944        Louis Jordan   

                                  track  rank  \
0                    Smoke On The Water     1   
1           Straighten Up And Fly Right     2   
2  Is You Is or Is You Ain't (Ma' Baby)     3   
3           I'm Wastin' My Tears on You     4   
4                          Ration Blues     5   

                                                link  \
0  /charts/country/video/1944/red-foley-smoke-on-...   
1  /charts/country/video/1944/the-king-cole-trio-...   
2  /charts/country/video/1944/louis-jordan-is-you...   
3  /charts/country/video/1944/tex-ritter-im-wasti...   
4  /charts/country/video/1944/louis-jordan-ration...   

                                              lyrics  
0  Smoke On The Water Ly

# Search Musicbrainz

In [192]:
small_df = df.head()
small_df

,index,track_id,year,artist,track,rank,link,lyrics
0,0,0,1944,Red Foley,Smoke On The Water,1,/charts/country/video/1944/red-foley-smoke-on-...,Smoke On The Water LyricsThere will be a sad d...
1,1,1,1944,The King Cole Trio,Straighten Up And Fly Right,2,/charts/country/video/1944/the-king-cole-trio-...,Straighten Up and Fly Right LyricsA buzzard to...
2,2,2,1944,Louis Jordan,Is You Is or Is You Ain't (Ma' Baby),3,/charts/country/video/1944/louis-jordan-is-you...,Is You Is Or Is You Ain’t (ma Baby) LyricsBing...
3,3,3,1944,Tex Ritter,I'm Wastin' My Tears on You,4,/charts/country/video/1944/tex-ritter-im-wasti...,List of Number One Country Songs Lyrics1944\nB...
4,4,4,1944,Louis Jordan,Ration Blues,5,/charts/country/video/1944/louis-jordan-ration...,"Ration Blues LyricsBaby, baby, baby\nWhat's wr..."


In [193]:
# https://datascience.stackexchange.com/a/89267
df['artist_id'] = pd.factorize(df['artist'])[0] + 1 # just don't want to start at 0

In [194]:
artists = df.artist.unique()
artists_head = artists
print(artists_head.size)

1177


In [195]:
df_unique_artists = df.groupby('artist', as_index=False).first()

In [197]:
artists_df = df_unique_artists.progress_apply(lambda row: search_artist_series(row["artist_id"], row["artist"]), axis=1)

  0%|          | 0/1177 [00:00<?, ?it/s]

In [198]:
artists_df.to_pickle("22-04-27-artists-pickle")

In [199]:
artists_df.columns = [i.replace('-', '_') for i in artists_df.columns]

1177

In [201]:
df['artist_appearances'] = df.groupby('artist_id')['artist_id'].transform('count')

In [207]:
artists_df = artists_df.rename(columns={"id": "mb_id"})

In [208]:
artists_df.applymap(str).to_sql("artists", lyrics_db, if_exists="replace")
df.to_sql("lyrics", lyrics_db, if_exists="replace")

7418